In [1]:
# -----------------------------------------------------------
# CELL 1: Import Required Libraries
# -----------------------------------------------------------

import re              # For pattern matching
import pandas as pd    # For structured data handling